In [1]:
import torch

torch.set_printoptions(sci_mode=False)

def square_indices(n):
    max_idx = 2 * (n+1)
    res = [[0, x] for x in range(max_idx -1, -1, -1)]\
        + [[x, 0] for x in range(1, max_idx)]\
        + [[max_idx -1, x] for x in range(1, max_idx)]\
        + [[x, max_idx -1] for x in range(max_idx -2, 0, -1)]
    return res

# I need to create a grid from a mesh
def mesh_grid(mesh, square_no):
    side = (square_no+1) * 2
    grid = torch.zeros(side, side)
    for sq_no in range(square_no+1):
        fill_square(mesh, grid, sq_no, square_no)
    return grid

def fill_square(mesh, grid, sq_no, square_no):
    print(sq_no, square_no)
    sq_ids =  square_indices(sq_no)
    offset = square_no - sq_no
    for no, (i, k) in enumerate(sq_ids):
        grid[i+offset, k+offset] = 3 **sq_no + no
    
mesh_grid(None, 5)

0 5
1 5
2 5
3 5
4 5
5 5


tensor([[254., 253., 252., 251., 250., 249., 248., 247., 246., 245., 244., 243.],
        [255.,  90.,  89.,  88.,  87.,  86.,  85.,  84.,  83.,  82.,  81., 286.],
        [256.,  91.,  34.,  33.,  32.,  31.,  30.,  29.,  28.,  27., 116., 285.],
        [257.,  92.,  35.,  14.,  13.,  12.,  11.,  10.,   9.,  54., 115., 284.],
        [258.,  93.,  36.,  15.,   6.,   5.,   4.,   3.,  28.,  53., 114., 283.],
        [259.,  94.,  37.,  16.,   7.,   2.,   1.,  14.,  27.,  52., 113., 282.],
        [260.,  95.,  38.,  17.,   8.,   3.,   4.,  13.,  26.,  51., 112., 281.],
        [261.,  96.,  39.,  18.,   9.,  10.,  11.,  12.,  25.,  50., 111., 280.],
        [262.,  97.,  40.,  19.,  20.,  21.,  22.,  23.,  24.,  49., 110., 279.],
        [263.,  98.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,  48., 109., 278.],
        [264.,  99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 277.],
        [265., 266., 267., 268., 269., 270., 271., 272., 273., 274., 275., 276.]])

In [19]:
import numpy as np
import torch
from src.polar_traversal import (
    Bridge,
    make_latitudes,
    get_start,
    gather_path,
    cumulative_distances,
)

def get_sample(pos, normed, path_points, path_normals):    
    i = 1
    while i < len(normed):
        d0, d1 = normed[i-1], normed[i]
        if pos <= d1:
            ratio = (pos - d0) / (d1 - d0) 
            pnt = ratio * path_points[i-1] + (1 - ratio) * path_points[i]
            nrm = ratio * path_normals[i-1] + (1 - ratio) * path_normals[i]
            return pnt, nrm, i
        i += 1
    raise (pos, normed, path)

def single_path_sample(path, layer_no, total_layers):
    cumulative = cumulative_distances(path['points'])
    normed =  cumulative / cumulative[-1]
    pos = layer_no / total_layers    
    pnt, nrm, _ = get_sample(pos, normed, 
                             path['points'], path['normals'])
    return pnt, nrm



def get_angles_series(n):
    return [x for x in range(0, n*2+1, 2)]

def get_shifted_latitudes(no):
    lats = make_latitudes(no)
    rot = len(lats) // 4
    return np.vstack([lats[rot:], lats[:rot]])

def get_sample(pos, normed, path_points, path_normals):    
    i = 1
    while i < len(normed):
        d0, d1 = normed[i-1], normed[i]
        if pos <= d1:
            ratio = (pos - d0) / (d1 - d0) 
            pnt = ratio * path_points[i-1] + (1 - ratio) * path_points[i]
            nrm = ratio * path_normals[i-1] + (1 - ratio) * path_normals[i]
            return pnt, nrm, i
        i += 1
    raise (pos, normed, path)

def single_path_sample(path, layer_no, total_layers):
    cumulative = cumulative_distances(path['points'])
    normed =  cumulative / cumulative[-1]
    pos = layer_no / total_layers    
    pnt, nrm, _ = get_sample(pos, normed, 
                             path['points'], path['normals'])
    return torch.tensor(pnt).float(), torch.tensor(nrm).float()

def square_indices(n):
    max_idx = 2 * (n+1)
    res = [[0, x] for x in range(max_idx -1, -1, -1)]\
        + [[x, 0] for x in range(1, max_idx)]\
        + [[max_idx -1, x] for x in range(1, max_idx)]\
        + [[x, max_idx -1] for x in range(max_idx -2, 0, -1)]
    return res

def shifted_square_indices(sq_no, square_no):    
    sq_ids =  square_indices(sq_no)
    ost = square_no - sq_no
    return [(i+ost, k+ost) for (i, k) in iter(sq_ids)]

class Blueprint:
    
    def __init__(self, path):
        self.path = path
        self.bridge = Bridge(path)
        
    def __repr__(self):
        return f'Blueprint: {self.path}'    

    def grid_sample(self, square_no):
        side = (square_no + 1) * 2
        points = torch.zeros(side, side, 3)
        normals = torch.zeros(side, side, 3)
        
        face_id, point, normal = get_start(self.bridge.mesh)
        for sq_no in range(square_no+1):
            side = (sq_no + 1) * 2
            angles_no = 4 * (sq_no + 1)
            latitudes = get_shifted_latitudes(angles_no)
            
            sq_ids = shifted_square_indices(sq_no, square_no)            
            for (r, c), latitude in zip(sq_ids, latitudes):
                path = gather_path(latitude, face_id, point, normal, self.bridge)
                pnt, nrm = single_path_sample(path, sq_no, square_no)
                points[r, c] = pnt
                normals[r, c] = nrm            
        return points, normals

path = './data/centered_3.stl'
square_no = 4
blueprint = Blueprint(path)
print(blueprint)

points, normals = blueprint.grid_sample(square_no)
points.shape, normals.shape

Blueprint: ./data/centered_3.stl


(torch.Size([10, 10, 3]), torch.Size([10, 10, 3]))

In [8]:
sides = get_angles_series(square_no)
square_no, sides

(4, [0, 2, 4, 6, 8])

In [13]:
torch.zeros(4, 8) #= 4
# 0 -  2
# 1 -  4
# 2 -  6
# 3 -  8
# 4 - 10
get_angles_series(0)

[0]

In [36]:
def square_indices(n):
    max_idx = 2 * (n+1)
    res = [[0, x] for x in range(max_idx -1, -1, -1)]\
        + [[x, 0] for x in range(1, max_idx)]\
        + [[max_idx -1, x] for x in range(1, max_idx)]\
        + [[x, max_idx -1] for x in range(max_idx -2, 0, -1)]
    return res

def shifted_square_indices(sq_no, square_no):    
    sq_ids =  square_indices(sq_no)
    ost = square_no - sq_no
    return [(i+ost, k+ost) for (i, k) in iter(sq_ids)]

sq_no, square_no = 0, 2
shifted_square_indices(sq_no, square_no)

[(2, 3), (2, 2), (3, 2), (3, 3)]

In [13]:
square_no = 4
sides = get_angles_series(square_no)
print(sides)
side = sides[-1]
for side in sides:
    angles_no = get_angles_no(side)
    print(angles_no)

[0, 2, 4, 6, 8]
4
12
20
28
36


In [17]:
lats = make_latitudes(12)
lats

array([[[ 0.0000000e+00,  0.0000000e+00],
        [ 2.0000000e+00,  0.0000000e+00]],

       [[ 0.0000000e+00,  0.0000000e+00],
        [ 1.7320508e+00,  1.0000000e+00]],

       [[ 0.0000000e+00,  0.0000000e+00],
        [ 9.9999994e-01,  1.7320508e+00]],

       [[ 0.0000000e+00,  0.0000000e+00],
        [-8.7422777e-08,  2.0000000e+00]],

       [[ 0.0000000e+00,  0.0000000e+00],
        [-1.0000001e+00,  1.7320508e+00]],

       [[ 0.0000000e+00,  0.0000000e+00],
        [-1.7320508e+00,  1.0000001e+00]],

       [[ 0.0000000e+00,  0.0000000e+00],
        [-2.0000000e+00, -1.7484555e-07]],

       [[ 0.0000000e+00,  0.0000000e+00],
        [-1.7320505e+00, -1.0000004e+00]],

       [[ 0.0000000e+00,  0.0000000e+00],
        [-9.9999982e-01, -1.7320509e+00]],

       [[ 0.0000000e+00,  0.0000000e+00],
        [ 2.3849761e-08, -2.0000000e+00]],

       [[ 0.0000000e+00,  0.0000000e+00],
        [ 9.9999982e-01, -1.7320509e+00]],

       [[ 0.0000000e+00,  0.0000000e+00],
        [ 1.

In [23]:
rot = len(lats) // 4
rot

3

In [24]:
lats.shape

(12, 2, 2)

In [25]:
lats[rot:].shape

(9, 2, 2)

In [26]:
lats[:rot].shape

(3, 2, 2)

In [30]:
import numpy as np

np.vstack([lats[rot:], lats[:rot]]).shape

(12, 2, 2)